In [ ]:
from nanopore_tcr_consensus.analysis import (
    set_mpl_params,
    split_ref_names_set_into_subsets,
    parse_raw_nanopore_qual_from_fastq_stats,
    parse_quantile_95_blast_id_from_self_homology_log,
    parse_merged_consensus_bam_filter_log,
    plot_blast_id_with_most_similar_tcr,
    plot_hist_subreads_per_umi_cluster,
    plot_box_blast_id_vs_subreads_umi_cluster,
    plot_hist_nt_too_short,
    plot_hist_nt_too_long,
    plot_hist_percent_alignments_above_blast_id,
    log_normalize_umi_counts,
    plot_umi_counts_hist,
    plot_log_transformed_umi_counts_hist,
    plot_v_gene_fold_change_over_input_barplot,
    plot_v_gene_fraction_missing_tcrs,
    filter_counts_on_log_umi_count_threshold,
    write_results_summary_txt,
    plot_plate_umi_counts,
)
from nanopore_tcr_consensus.region_split import generate_regions_set_from_ref_fa

import os
import pandas as pd
import glob
import re

import matplotlib as mpl
from matplotlib.cm import viridis

viridis.set_bad("lightgrey")
mpl, mylines = set_mpl_params(mpl=mpl)

In [ ]:
nanopore_project_dir = ""
nano_tcr_name = "nano_tcr"
internal_basecalling = True
plot_plates = False

blast_id_ylim_max = 350
umi_count_xlim_max = 900
umi_count_bin_size = 10
umi_count_ylim_max = 0.1
log_umi_count_xlim_max = 8
log_umi_count_bin_size = 0.25
log_umi_count_ylim_max = 1.0
most_similar_blast_id_threshold = 0.99925


libraries_csv = os.path.join(nanopore_project_dir, "libraries.csv")
nanopore_project_outs_dir = os.path.join(nanopore_project_dir, "outs")
os.mkdir(nanopore_project_outs_dir)
libraries_df = pd.read_csv(libraries_csv)
libraries_df.to_csv(os.path.join(nanopore_project_outs_dir, "libraries.csv"))
library_list = libraries_df.loc[:, "barcode"].tolist()
if internal_basecalling:
    library_list = ["concatenated_" + library for library in library_list]
library_name_list = libraries_df.loc[:, "library_name"].tolist()
if any([" " in library_name for library_name in library_name_list]):
    raise Exception("white space is not allowed in library names!")
library_ref_id_list = libraries_df.loc[:, "ref_library_name"].tolist()
library_ref_id_list = [
    False if pd.isna(library_ref_id) else library_ref_id
    for library_ref_id in library_ref_id_list
]
log_umi_counts_filter_threshold_list = libraries_df.loc[
    :, "log_umi_counts_filter_threshold"
].tolist()
if len(glob.glob(os.path.join(nanopore_project_dir, "*.fa"))) > 1:
    raise Exception(
        "There is more than reference .fa file in the nanopore_project_dir:",
        nanopore_project_dir,
    )
reference_name = glob.glob(os.path.join(nanopore_project_dir, "*.fa"))[0].split(".fa")[
    0
]
reference = os.path.join(nanopore_project_dir, reference_name + ".fa")
ref_names_set = generate_regions_set_from_ref_fa(reference=reference)
non_n_ref_names_set, full_n_ref_names_set, cdr3j_n_ref_names_set, v_n_ref_names_set = (
    split_ref_names_set_into_subsets(ref_names_set=ref_names_set)
)

In [ ]:
# custom_tcr_set_dict = None
gil_plate_1_duplicate_1_ref_set = set()
pattern = r"^[A-H](?:[1-9]|1[0-9]|2[0-4])$"
for ref in non_n_ref_names_set:
    if (
        ref.endswith("GILGFVFTL")
        and ref.startswith("1_1")
        and re.match(pattern, ref.split("_")[2])
    ):
        gil_plate_1_duplicate_1_ref_set.add(ref)

custom_tcr_set_dict = {"gil_plate_1_tcrs": gil_plate_1_duplicate_1_ref_set}

In [ ]:
for (
    library,
    library_name,
    library_ref_id,
    log_umi_counts_filter_threshold,
) in zip(
    library_list,
    library_name_list,
    library_ref_id_list,
    log_umi_counts_filter_threshold_list,
):
    print("Processing:", library_name)
    library_dir = os.path.join(
        nanopore_project_dir, "fastq_pass", nano_tcr_name, library
    )
    umi_counts_csv = os.path.join(library_dir, "counts", "umi_consensus_counts.csv")
    logs_dir = os.path.join(library_dir, "logs")
    fastq_stats_before_filtering_txt = os.path.join(
        logs_dir, library + "_trimmed_stats_before_vsearch_filtering.txt"
    )
    number_subreads_blast_id_csv = os.path.join(
        logs_dir, "merged_consensus_number_of_subreads_blast_id.csv"
    )
    merged_consensus_bam_filter_log = os.path.join(
        logs_dir, "merged_consensus_bam_filter.log"
    )
    blast_id_csv = os.path.join(logs_dir, "merged_consensus_blast_id.csv")
    nt_too_short_csv = os.path.join(logs_dir, "merged_consensus_nt_too_short.csv")
    region_nt_too_short_csv = os.path.join(
        logs_dir, "merged_consensus_region_nt_too_short.csv"
    )
    nt_too_long_csv = os.path.join(logs_dir, "merged_consensus_nt_too_long.csv")
    region_nt_too_long_csv = os.path.join(
        logs_dir, "merged_consensus_region_nt_too_long.csv"
    )
    self_homology_log = os.path.join(
        nanopore_project_dir,
        reference_name + "_ref_homology_out_generate_region_split_dict.log",
    )
    outs_dir = os.path.join(nanopore_project_outs_dir, library + "_" + library_name)
    os.mkdir(outs_dir)
    tcr_refs_df = pd.read_csv(os.path.join(nanopore_project_dir, "tcr_refs_df.csv"))
    
    if library_ref_id:
        library_non_n_ref_names_set = set(
            [ref for ref in non_n_ref_names_set if library_ref_id in ref]
        )
        library_full_n_ref_names_set = set(
            [ref for ref in full_n_ref_names_set if library_ref_id in ref]
        )
        library_cdr3j_n_ref_names_set = set(
            [ref for ref in cdr3j_n_ref_names_set if library_ref_id in ref]
        )
        library_v_n_ref_names_set = set(
            [ref for ref in v_n_ref_names_set if library_ref_id in ref]
        )
    else:
        library_non_n_ref_names_set = non_n_ref_names_set
        library_full_n_ref_names_set = full_n_ref_names_set
        library_cdr3j_n_ref_names_set = cdr3j_n_ref_names_set
        library_v_n_ref_names_set = v_n_ref_names_set

    raw_nanopore_qual = parse_raw_nanopore_qual_from_fastq_stats(
        fastq_stats_before_filtering_txt=fastq_stats_before_filtering_txt
    )
    (
        minimal_blast_id,
        max_too_many_nt,
        max_too_few_nt,
        percent_too_few_nt,
        percent_too_many_nt,
        percent_correct_overlap_length,
    ) = parse_merged_consensus_bam_filter_log(
        merged_consensus_bam_filter_log=merged_consensus_bam_filter_log
    )
    quantile_95_blast_id = parse_quantile_95_blast_id_from_self_homology_log(
        self_homology_log=self_homology_log
    )

    print("Plotting blast_id_with_most_similar_tcr")

    plot_blast_id_with_most_similar_tcr(
        nanopore_project_dir=nanopore_project_dir,
        xlim_min=0.96,
        ylim_max=blast_id_ylim_max,
        library_ref_id=library_ref_id,
        minimal_blast_id=minimal_blast_id,
        quantile_95_blast_id=quantile_95_blast_id,
        raw_nanopore_qual=raw_nanopore_qual,
        title=library_name,
        save_suffix=None,
        outs_dir=outs_dir,
    )

    plot_blast_id_with_most_similar_tcr(
        nanopore_project_dir=nanopore_project_dir,
        xlim_min=0.99,
        ylim_max=blast_id_ylim_max,
        library_ref_id=library_ref_id,
        minimal_blast_id=minimal_blast_id,
        quantile_95_blast_id=quantile_95_blast_id,
        raw_nanopore_qual=raw_nanopore_qual,
        title=library_name,
        save_suffix="zoom_in",
        outs_dir=outs_dir,
    )

    print("Plotting hist_subreads_per_umi_cluster")
    plot_hist_subreads_per_umi_cluster(
        number_subreads_blast_id_csv=number_subreads_blast_id_csv,
        title=library_name,
        outs_dir=outs_dir,
    )

    print("Plotting box_blast_id_vs_subreads_umi_cluster")
    plot_box_blast_id_vs_subreads_umi_cluster(
        number_subreads_blast_id_csv=number_subreads_blast_id_csv,
        minimal_blast_id=minimal_blast_id,
        title=library_name,
        outs_dir=outs_dir,
    )

    print("Plotting hist_nt_too_short")
    plot_hist_nt_too_short(
        nt_too_short_csv=nt_too_short_csv,
        tcr_refs_df=tcr_refs_df,
        max_too_few_nt=max_too_few_nt,
        percent_too_few_nt=percent_too_few_nt,
        title=library_name,
        outs_dir=outs_dir,
    )

    print("Plotting hist_nt_too_long")
    plot_hist_nt_too_long(
        nt_too_long_csv=nt_too_long_csv,
        max_too_many_nt=max_too_many_nt,
        percent_too_many_nt=percent_too_many_nt,
        title=library_name,
        outs_dir=outs_dir,
    )

    print("Plotting hist_percent_alignments_above_blast_id")
    plot_hist_percent_alignments_above_blast_id(
        blast_id_csv=blast_id_csv,
        minimal_blast_id=minimal_blast_id,
        quantile_95_blast_id=quantile_95_blast_id,
        percent_correct_overlap_length=percent_correct_overlap_length,
        title=library_name,
        outs_dir=outs_dir,
    )

    counts_df = pd.read_csv(umi_counts_csv)
    counts_df = log_normalize_umi_counts(counts_df=counts_df)

    print("Plotting umi_counts_hists")
    plot_umi_counts_hist(
        counts_df=counts_df,
        xlim_max=umi_count_xlim_max,
        bin_size=umi_count_bin_size,
        ylim_max=umi_count_ylim_max,
        title=library_name,
        plot_nb_dist_fit=True,
        plot_percentiles=True,
        save_suffix=None,
        outs_dir=outs_dir,
    )

    plot_umi_counts_hist(
        counts_df=counts_df,
        xlim_max=umi_count_xlim_max,
        bin_size=umi_count_bin_size,
        ylim_max=umi_count_ylim_max,
        title=library_name,
        plot_nb_dist_fit=False,
        plot_percentiles=True,
        save_suffix="without_nb_dist_fit",
        outs_dir=outs_dir,
    )

    print("Plotting log_transformed_umi_counts_hists")
    plot_log_transformed_umi_counts_hist(
        counts_df=counts_df,
        xlim_max=log_umi_count_xlim_max,
        bin_size=log_umi_count_bin_size,
        ylim_max=log_umi_count_ylim_max,
        custom_tcr_set=None,
        label_custom_tcr_set=None,
        density=True,
        plot_normal_dist_fit=True,
        log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
        plot_percentiles=True,
        most_similar_blast_id_threshold=None,
        title=library_name,
        save_suffix=None,
        nanopore_project_dir=nanopore_project_dir,
        outs_dir=outs_dir,
    )

    plot_log_transformed_umi_counts_hist(
        counts_df=counts_df,
        xlim_max=log_umi_count_xlim_max,
        bin_size=log_umi_count_bin_size,
        ylim_max=log_umi_count_ylim_max,
        custom_tcr_set=None,
        label_custom_tcr_set=None,
        density=True,
        plot_normal_dist_fit=False,
        log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
        plot_percentiles=True,
        most_similar_blast_id_threshold=None,
        title=library_name,
        save_suffix="without_normal_dist_fit",
        nanopore_project_dir=nanopore_project_dir,
        outs_dir=outs_dir,
    )

    plot_log_transformed_umi_counts_hist(
        counts_df=counts_df,
        xlim_max=log_umi_count_xlim_max,
        bin_size=log_umi_count_bin_size,
        ylim_max=log_umi_count_ylim_max,
        custom_tcr_set=None,
        label_custom_tcr_set=None,
        density=True,
        plot_normal_dist_fit=False,
        log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
        plot_percentiles=True,
        most_similar_blast_id_threshold=most_similar_blast_id_threshold,
        title=library_name,
        save_suffix="colored_by_most_similar_blast_id",
        nanopore_project_dir=nanopore_project_dir,
        outs_dir=outs_dir,
    )

    if counts_df.loc[:, "TCR"].isin(library_full_n_ref_names_set).any():
        plot_log_transformed_umi_counts_hist(
            counts_df=counts_df,
            xlim_max=log_umi_count_xlim_max,
            bin_size=log_umi_count_bin_size,
            ylim_max=log_umi_count_ylim_max,
            custom_tcr_set=library_full_n_ref_names_set,
            label_custom_tcr_set="full-length synthetic\negative control TCRs",
            density=True,
            plot_normal_dist_fit=False,
            log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
            plot_percentiles=True,
            most_similar_blast_id_threshold=None,
            title=library_name,
            save_suffix="colored_by_full_n_refs",
            nanopore_project_dir=nanopore_project_dir,
            outs_dir=outs_dir,
        )

    if counts_df.loc[:, "TCR"].isin(library_cdr3j_n_ref_names_set).any():
        plot_log_transformed_umi_counts_hist(
            counts_df=counts_df,
            xlim_max=log_umi_count_xlim_max,
            bin_size=log_umi_count_bin_size,
            ylim_max=log_umi_count_ylim_max,
            custom_tcr_set=library_cdr3j_n_ref_names_set,
            label_custom_tcr_set="CDR3-J synthetic\negative control TCRs",
            density=True,
            plot_normal_dist_fit=False,
            log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
            plot_percentiles=True,
            most_similar_blast_id_threshold=None,
            title=library_name,
            save_suffix="colored_by_cdr3j_n_refs",
            nanopore_project_dir=nanopore_project_dir,
            outs_dir=outs_dir,
        )

    if counts_df.loc[:, "TCR"].isin(library_v_n_ref_names_set).any():
        plot_log_transformed_umi_counts_hist(
            counts_df=counts_df,
            xlim_max=log_umi_count_xlim_max,
            bin_size=log_umi_count_bin_size,
            ylim_max=log_umi_count_ylim_max,
            custom_tcr_set=library_v_n_ref_names_set,
            label_custom_tcr_set="V gene synthetic\negative control TCRs",
            density=True,
            plot_normal_dist_fit=False,
            log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
            plot_percentiles=True,
            most_similar_blast_id_threshold=None,
            title=library_name,
            save_suffix="colored_by_v_n_refs",
            nanopore_project_dir=nanopore_project_dir,
            outs_dir=outs_dir,
        )

    if custom_tcr_set_dict:
        for label_custom_tcr_set, custom_tcr_set in custom_tcr_set_dict.items():
            plot_log_transformed_umi_counts_hist(
                counts_df=counts_df,
                xlim_max=log_umi_count_xlim_max,
                bin_size=log_umi_count_bin_size,
                ylim_max=log_umi_count_ylim_max,
                custom_tcr_set=custom_tcr_set,
                label_custom_tcr_set=label_custom_tcr_set,
                density=True,
                plot_normal_dist_fit=False,
                log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
                plot_percentiles=True,
                most_similar_blast_id_threshold=None,
                title=library_name,
                save_suffix="colored_by_" + label_custom_tcr_set,
                nanopore_project_dir=nanopore_project_dir,
                outs_dir=outs_dir,
            )

            plot_log_transformed_umi_counts_hist(
                counts_df=counts_df,
                xlim_max=log_umi_count_xlim_max,
                bin_size=log_umi_count_bin_size,
                ylim_max=log_umi_count_ylim_max,
                custom_tcr_set=custom_tcr_set,
                label_custom_tcr_set=label_custom_tcr_set,
                density=True,
                plot_normal_dist_fit=False,
                log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
                plot_percentiles=False,
                most_similar_blast_id_threshold=None,
                title=library_name,
                save_suffix="colored_by_" + label_custom_tcr_set + "_no_percentiles",
                nanopore_project_dir=nanopore_project_dir,
                outs_dir=outs_dir,
            )

    print("Plotting v_gene_fold_change_over_input_barplot")
    plot_v_gene_fold_change_over_input_barplot(
        counts_df=counts_df,
        tcr_refs_df=tcr_refs_df,
        trav_col="TRAV_IMGT_allele_collapsed",
        trbv_col="TRBV_IMGT_allele_collapsed",
        title=library_name,
        outs_dir=outs_dir,
    )

    print("Plotting v_gene_fraction_missing_tcrs")
    plot_v_gene_fraction_missing_tcrs(
        counts_df=counts_df,
        tcr_refs_df=tcr_refs_df,
        non_n_ref_names_set=library_non_n_ref_names_set,
        region_name_col="TCR",
        trav_col="TRAV_IMGT_allele_collapsed",
        trbv_col="TRBV_IMGT_allele_collapsed",
        title=library_name,
        outs_dir=outs_dir,
    )

    counts_df = filter_counts_on_log_umi_count_threshold(
        counts_df=counts_df,
        log_umi_counts_filter_threshold=log_umi_counts_filter_threshold,
    )

    plot_log_transformed_umi_counts_hist(
        counts_df=counts_df,
        xlim_max=log_umi_count_xlim_max,
        bin_size=log_umi_count_bin_size,
        ylim_max=log_umi_count_ylim_max,
        custom_tcr_set=None,
        label_custom_tcr_set=None,
        density=True,
        plot_normal_dist_fit=False,
        log_umi_counts_filter_threshold=None,
        plot_percentiles=True,
        most_similar_blast_id_threshold=None,
        title=library_name,
        save_suffix="after_log_umi_count_filtering",
        nanopore_project_dir=nanopore_project_dir,
        outs_dir=outs_dir,
    )

    print("Writing results_summary_txt")
    write_results_summary_txt(
        counts_df=counts_df,
        merged_consensus_bam_filter_log=merged_consensus_bam_filter_log,
        region_name_col="TCR",
        library_name=library_name,
        non_n_ref_names_set=library_non_n_ref_names_set,
        outs_dir=outs_dir,
    )

    if plot_plates:
        print("Plotting plate_umi_counts")
        plot_plate_umi_counts(
            counts_df=counts_df,
            non_n_ref_names_set=library_non_n_ref_names_set,
            region_name_col="TCR",
            outs_dir=outs_dir,
        )

    print("Writing filtered_umi_count csv")
    counts_df.to_csv(
        os.path.join(outs_dir, library + "_" + library_name + "_filtered_umi_count.csv")
    )
    print("\n\n")

Processing: vdjdb_technical_duplicate_1
Plotting blast_id_with_most_similar_tcr
Plotting hist_subreads_per_umi_cluster
Plotting box_blast_id_vs_subreads_umi_cluster
Plotting hist_nt_too_short
Plotting hist_nt_too_long
Plotting hist_percent_alignments_above_blast_id
Plotting umi_counts_hists
KS Statistic: 0.09697589911308102, p-value: 2.1457867866486665e-30
Plotting log_transformed_umi_counts_hists
KS Statistic: 0.20663946070371503, p-value: 1.8240884995187867e-137
Plotting v_gene_fold_change_over_input_barplot
Plotting v_gene_fraction_missing_tcrs
Writing results_summary_txt
Writing filtered_umi_count csv



Processing: vdjdb_technical_duplicate_2
Plotting blast_id_with_most_similar_tcr
Plotting hist_subreads_per_umi_cluster
Plotting box_blast_id_vs_subreads_umi_cluster
Plotting hist_nt_too_short
Plotting hist_nt_too_long
Plotting hist_percent_alignments_above_blast_id
Plotting umi_counts_hists
KS Statistic: 0.10598730428168571, p-value: 7.629108194013631e-36
Plotting log_transformed_u

In [ ]:
region_nt_too_short_df = pd.read_csv(region_nt_too_short_csv)

In [ ]:
region_nt_too_short_df.loc[:, "region"].value_counts()[:60]

In [ ]:
region_nt_too_long_df = pd.read_csv(region_nt_too_long_csv)

In [ ]:
region_nt_too_long_df.loc[:, "region"].value_counts()[:60]